In [1]:
import numpy  as np
import pandas as pd
import requests
import time 
import re
from   bs4    import BeautifulSoup 
import string
import os

### Send a GET request to the server.

In [2]:
#For other artists lyrics, just change the URL and find&replace the artist name in code!
the_artist = 'hammerfall'
URL        = 'https://www.lyrics.com/artist/Hammerfall/212623'

#Disguising as a browser. Submit headers that the script looks like a web browser
headers    = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

response   = requests.get(URL, headers=headers)

print(f'Check the connection (Code 200: Get successful!):\nResponse Status Code: {response.status_code}\n')

Check the connection (Code 200: Get successful!):
Response Status Code: 200



In [3]:
response.text[:200] #check the text roughly.

'<!doctype html>\n<html lang="en-US">\n<head>\n<meta name="theme-color" content="#830C66"/>\n\n<meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<title>Hammerfall Lyrics,'

### Download the artists main webpage's HTML source-code in str format and save in txt file.

In [4]:
# Save the file to local disks as .txt 

with open(file=f"lyrics/{the_artist}_main.txt", mode="w") as my_file:
    my_file.write(response.text)   

In [5]:
# Read the saved file

with open(file=f"lyrics/{the_artist}_main.txt", mode="r") as file:
    the_artist_text = file.read()       #str all the text
    #text = file.readlines() #read all lines and create a list
    #text = file.readline()  #use to loop over lines

### Scrap the HTML code by BeatifulSoup lib. 

In [6]:
the_artist_soup=BeautifulSoup(markup=the_artist_text, features="html.parser")

#### Find raw links of song lyrics

In [7]:
raw_link_list = the_artist_soup.find_all(name="a", href=True)
print(f'Number of raw links: {len(raw_link_list)}')

Number of raw links: 738


#### find song links and add them in 'song_link_list'

In [8]:
song_link_list=[]
for link in raw_link_list:
    if link['href'].startswith('/lyric'):
        song_link_list.append("https://www.lyrics.com"+link['href'])
print(f'Number of links: {len(song_link_list)}')

Number of links: 458


#### In main page check every song pages (i) and download every page in different file

In [9]:
def download_songs():    
    number_songs = song_link_list
    print(f'                             ==================={len(number_songs)} songs are in process!=================== ....................................................................')
    for i, songlink in enumerate(song_link_list):
        url_link = songlink
        headers_link = {'User-agent': 'Mozilla/5.0 (X11; Linux i686; rv:2.0b10) Gecko/20100101 Firefox/4.0b10'}
        print(f'===============  Downloading from ===>{songlink}')
        response_link = requests.get(url=url_link, headers=headers_link)
        print(f'===============  Saving')

        with open(file=f"lyrics/{the_artist}/{the_artist}_lyric_{i}.txt", mode="w") as the_file:
            the_file.write(response_link.text)   
        timer=np.random.random()
        print(f'===============  Waiting {round(timer,2)} seconds - PROCESS: %{round(((i+1)/len(number_songs)),2)}')
        time.sleep(timer)  # waiting time in seconds
    print('                                     ===================FINISHED===================')

In [10]:
# Create folders to download artist's lyric webpages
parent_dir_path = !pwd
parent_dir_path = parent_dir_path[0]
directory = parent_dir_path + '/lyrics/' + the_artist

if os.path.exists(directory):
    print(f"The directory \'{directory}\' exists.")
else:
    os.mkdir(directory)
    print(f"The directory \'{directory}\' created.")
    download_songs()

The directory '/home/elendar/ds_projects/song_artist_prediction/lyrics/hammerfall' exists.


#### Read files and extract lyrics with song names

In [11]:
#pattern2 = '[<][<\w\d =\":/\.\#;]*[>]'
all_sentences          = []
song_names             = []
amount_sentences_list  = []

for i in range(len(song_link_list)): 
    print(f'{i}/{len(song_link_list)}')
    with open(file=f"lyrics/{the_artist}/{the_artist}_lyric_{i}.txt", mode="r") as file:            
        print(f'open file: {the_artist}_lyric_{i}.txt')
        the_artist_text = file.read()       #str all the text

    the_artist_soup1  = BeautifulSoup(markup = the_artist_text, features='html.parser')
    raw_ly            = the_artist_soup1.find(name='pre', attrs={'class':"lyric-body"})
    raw_song_names    = the_artist_soup1.find(name='h1', attrs={'class':'lyric-title', 'id':"lyric-title-text"})
    
    if raw_ly is None:
        print('Empty page is detected!!! Continuing to next page!')
        continue
        
    ly = raw_ly.get_text()
    ly = ly.lower()
    ly = re.sub('\n\n','\n', ly)
    ly = ly.replace('-', ' ')
    #print([ch for ch in ly if ch not in string.punctuation])
    ly="".join([ch for ch in ly if ch not in string.punctuation]) # if string.punctuation gives all characters that i want to take out
    ly = re.split('\n', ly)                                       # check all characters in ly, if they are not in special 
                                                                  # characters join them together
    if len(ly) < 3:
        print('Empty page is detected!!! Continuing to next page!')
        continue
        
    all_sentences.extend(ly)
    print(f'Song name: {raw_song_names.get_text()}..............has {len(ly)} lines')
    for k in range(len(ly)):
        song_names.append(raw_song_names.get_text())

0/458
open file: hammerfall_lyric_0.txt
Song name: Hector's Hymn..............has 34 lines
1/458
open file: hammerfall_lyric_1.txt
Song name: Heeding the Call..............has 28 lines
2/458
open file: hammerfall_lyric_2.txt
Song name: Any Means Necessary..............has 42 lines
3/458
open file: hammerfall_lyric_3.txt
Song name: Riders of the Storm..............has 31 lines
4/458
open file: hammerfall_lyric_4.txt
Song name: Crimson Thunder..............has 26 lines
5/458
open file: hammerfall_lyric_5.txt
Song name: Renegade..............has 34 lines
6/458
open file: hammerfall_lyric_6.txt
Song name: Blood Bound..............has 44 lines
7/458
open file: hammerfall_lyric_7.txt
Song name: Last Man Standing..............has 63 lines
8/458
open file: hammerfall_lyric_8.txt
Song name: Fury of the Wild..............has 30 lines
9/458
open file: hammerfall_lyric_9.txt
Song name: Always Will Be..............has 31 lines
10/458
open file: hammerfall_lyric_10.txt
Song name: Steel Meets Steel..

In [12]:
catalog={'song_names':song_names,
        'sentences': all_sentences}

In [13]:
df = pd.DataFrame(catalog, index=song_names)

In [14]:
df

,song_names,sentences
Hector's Hymn,Hector's Hymn,i rose from the ashes the remains of a time se...
Hector's Hymn,Hector's Hymn,the warrior of eden found by honor these two w...
Hector's Hymn,Hector's Hymn,on and on and on with glory he came he saw he ...
Hector's Hymn,Hector's Hymn,he gathered his armies true he leads his templ...
Hector's Hymn,Hector's Hymn,they sought out the dragon made the enemy blee...
...,...,...
Destined for Glory [2020 Remix],Destined for Glory [2020 Remix],but fight without honor and youre destined to
Destined for Glory [2020 Remix],Destined for Glory [2020 Remix],fall
Destined for Glory [2020 Remix],Destined for Glory [2020 Remix],fight with your heart and youre destined for
Destined for Glory [2020 Remix],Destined for Glory [2020 Remix],glory


In [15]:
df['sentences'].isna().sum()

0

In [16]:
df['song_names'].isna().sum()

0

In [17]:
df.to_csv(f'{the_artist}.csv')